In [1]:
# %load_ext autoreload
# %autoreload 2

In [2]:
cd ..

/mnt/nfs-data/users/anhvth5/projects/guidance


/home/ubuntu/miniconda3/envs/py39/lib/python3.9/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [ ]:
from speedy import *
from guidance import *

os.environ['CUDA_VISIBLE_DEVICES'] = '0'
# qwen = models.prepare_model_guidance('../kilm-finetuning/output/q14_all_chat_data_v3_more_data_from_v1/')
qwen = models.prepare_model_guidance()

qas = []
for path in glob('../tvpl_crawler/data/tvpl_qa_giaothong/qa/*'):
    data = load_by_ext(path)
    for q in data['questions']:
        qas.append(json.dumps({'question': q['question'], 'answer': q['answer']}, ensure_ascii=False))





examples = pd.read_csv('/tmp/qa.csv', names=['input:', 'output:'])
examples
import guidance
@guidance
def generate_short_answer(lm):
    with system():
        lm = qwen + '''
Please restructure the provided answer according to these guidelines:
- Extract short and succinct quote to answer the user question

Input data:

{"question": "Từ ngày 27/11/2023, thủ tục cấp giấy xác nhận phù hợp về tuyển dụng, cung ứng thuyền viên hàng hải được thực hiện thế nào?", "answer": ["Căn cứ theo quy định tại khoản 7 Điều 3 Nghị định 74/2023/NĐ-CP sửa đổi bổ sung sửa đổi, bổ sung Điều 13 Nghị định 29/2017/NĐ-CP quy định về trình tự, thủ tục cấp giấy xác nhận phù hợp về tuyển dụng, cung ứng thuyền viên hàng hải được thực hiện theo các bước như sau:", "Bước 1: Nộp hồ sơ", "Tổ chức tuyển dụng, cung ứng thuyền viên hàng hải gửi trực tiếp hoặc qua hệ thống bưu chính hoặc qua hệ thống dịch vụ công trực tuyến đến Chi cục Hàng hải Việt Nam 01 bộ hồ sơ. Hồ sơ bao gồm:", "- Tờ khai theo Mẫu số 04 tại đây.", "- Bản sao có chứng thực hoặc bản sao kèm bản chính để đối chiếu hoặc bản sao điện tử hoặc bản sao điện tử từ sổ gốc hoặc bản sao điện tử có chứng thực từ bản chính giấy phép đưa thuyền viên đi làm việc ở nước ngoài hoặc giấy phép cho thuê hoặc cho thuê lại thuyền viên làm việc trên tàu biển dưới hình thức cho thuê, cho thuê lại lao động hoặc đưa người lao động đi làm việc ở nước ngoài theo quy định của pháp luật;", "- Bản sao có chứng thực hoặc bản sao kèm bản chính để đối chiếu hoặc bản sao điện tử hoặc bản sao điện tử từ sổ gốc hoặc bản sao điện tử có chứng thực từ bản chính Giấy chứng nhận phù hợp hệ thống quản lý chất lượng theo tiêu chuẩn ISO 9001 và Quy định 1.4 của Công ước MLC về dịch vụ tuyển dụng và cung ứng thuyền viên.", "Bước 2: Tiếp nhận hồ sơ, cấp giấy xác nhận", "Chi cục Hàng hải Việt Nam tiếp nhận hồ sơ, nếu hồ sơ chưa hợp lệ thì trong thời hạn 03 ngày làm việc kể từ ngày nhận được hồ sơ, Chi cục Hàng hải Việt Nam có văn bản thông báo và hướng dẫn tổ chức tuyển dụng, cung ứng thuyền viên hàng hải hoàn thiện hồ sơ theo quy định tại Nghị định 29/2017/NĐ-CP.", "Nếu hồ sơ hợp lệ thì trong thời hạn 03 ngày làm việc kể từ ngày nhận được hồ sơ, Chi cục Hàng hải Việt Nam phải cấp Giấy xác nhận theo Mẫu 05 quy định tại Phụ lục ban hành kèm theo Nghị định 29/2017/NĐ-CP; trường hợp không cấp phải có văn bản trả lời và nêu rõ lý do.", "Bước 3: Thông báo thông tin", "Chi cục Hàng hải Việt Nam thông báo thông tin về tổ chức tuyển dụng, cung ứng thuyền viên hàng hải được cấp Giấy xác nhận trên Cổng thông tin điện tử của Chi cục Hàng hải Việt Nam.", "Từ ngày 27/11/2023, thủ tục cấp giấy xác nhận phù hợp về tuyển dụng, cung ứng thuyền viên hàng hải được thực hiện thế nào? (Hình từ internet)"]},
Response :
```quote
Từ ngày 27/11/2023, thủ tục cấp giấy xác nhận phù hợp về tuyển dụng, cung ứng thuyền viên hàng hải bao gồm ba bước chính: Nộp hồ sơ, tiếp nhận hồ sơ và cấp giấy xác nhận, và thông báo thông tin.
```
'''
    return lm

app = qwen + generate_short_answer()
q = qas[2]
with user():
    lm = app + q
with assistant():
    temp= lm +'```quote\n'+gen('quote', suffix='```')



CPU times: user 6.96 s, sys: 571 ms, total: 7.54 s
Wall time: 7.53 s


In [59]:
with user():
    question = eval(q)['question']
    lm = qwen + 'Please answer the question given document: \nQuestion:{}\nDocument:{}'.format(question, temp['quote'])
with assistant():
    lm += 'Kết luận:' + gen()

In [26]:
# qas[-1]

In [27]:
# %%time
# with user():
#     lm = app + qas[1]
# with assistant():
#     lm += gen()